# Representing Assembly and Transformation Activities in Build Compilation

### Testing adding assembly activity and linking to composites

In [1]:
import sbol2

from sbol2build import (
    translate_abstract_to_plasmids,
    golden_gate_assembly_plan,
    rebase_restriction_enzyme,
    copy_sequences,
)

plasmid_collection = sbol2.Document()
sbh = sbol2.PartShop("https://synbiohub.org")
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/CIDARMoCloPlasmidsKit_collection/1/b7fdc21c6601a61d3166073a9e50f2c3843e1df5/share",
    plasmid_collection,
)
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloParts/CIDARMoCloParts_collection/1/038feff5f9c6c9b35cc95586ed6b14ec9bd3fea5/share",
    plasmid_collection,
)

abstract_design_doc = sbol2.Document()
abstract_design_doc.read("tests/test_files/moclo_parts_circuit.xml")

backbone_doc = sbol2.Document()
sbh.pull(
    "https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/DVK_AE/1/647c5b2458567dcce6b0a37178d352b8ffa9a7fe/share",
    backbone_doc,
)

mocloplasmid_list = translate_abstract_to_plasmids(
    abstract_design_doc, plasmid_collection, backbone_doc
)

# run assembly
part_documents = []
restriction_enzyme = rebase_restriction_enzyme("BsaI")
assembly_plan = sbol2.ModuleDefinition()

for mocloPlasmid in mocloplasmid_list:
    new_doc = sbol2.Document()
    mocloPlasmid.definition.copy(new_doc)
    copy_sequences(mocloPlasmid.definition, new_doc, plasmid_collection)
    part_documents.append(new_doc)

assembly_doc = sbol2.Document()
assembly_obj = golden_gate_assembly_plan(
    "assembly_plan", part_documents, backbone_doc, "BsaI", assembly_doc
)

composites = assembly_obj.run()

found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_AB/1 with ['Fusion_Site_B', 'Fusion_Site_A']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_EB/1 with ['Fusion_Site_E', 'Fusion_Site_B']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_FB/1 with ['Fusion_Site_B', 'Fusion_Site_F']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/J23100/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pJ23100_GB/1 with ['Fusion_Site_G', 'Fusion_Site_B']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/B0034/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pB0034_BC/1 with ['Fusion_Site_B', 'Fusion_Site_C']
found: https://synbiohub.org/user/Gon/CIDARMoCloParts/E0030_yfp/1 in https://synbiohub.org/user/Gon/CIDARMoCloPlasmidsKit/pE0030_CD/1 with ['F

## Bacterial Transformation Draft

In [2]:
# Pull chassis from sbh
chassis_doc = sbol2.Document()

sbh.pull(
    "https://synbiohub.org/user/Gon/Chassis/Ecoli_DH5a/1/52b575c09496ebb3e2ef9e4c272c9e733134a874/share",
    chassis_doc,
)

print(chassis_doc)

chassis = chassis_doc.moduleDefinitions[0]
plasmid = composites[0][0]

Design........................0
Build.........................0
Test..........................0
Analysis......................0
ComponentDefinition...........0
ModuleDefinition..............1
Model.........................0
Sequence......................0
Collection....................0
Activity......................0
Plan..........................0
Agent.........................0
Attachment....................0
CombinatorialDerivation.......0
Implementation................0
SampleRoster..................0
Experiment....................0
ExperimentalData..............0
Annotation Objects............0
---
Total: .........................1



In [3]:
transformation_activity = sbol2.Activity(f"transform_{chassis.name}")
transformation_activity.name = "Bacterial Tranformation"
transformation_activity.types = "http://sbols.org/v2#build"

chassis_usage = sbol2.Usage(
    uri=f"{chassis.name}_chassis_source",
    entity=chassis.identity,
    role="http://sbols.org/v2#build",
)

plasmid_usage = sbol2.Usage(
    uri=f"{plasmid.name}_plasmid_source",
    entity=plasmid.identity,
    role="http://sbols.org/v2#build",
)

transformation_activity.usages = [chassis_usage, plasmid_usage]

new_strain = sbol2.ModuleDefinition(f"{chassis.name}_with_{plasmid.name}")
chassis_module = sbol2.Module(uri=f"{chassis.name}_chassis")
chassis_module.definition = chassis.identity
plasmid_functional_component = sbol2.FunctionalComponent(
    uri=f"{plasmid.name}_engineered_plasmid"
)
plasmid_functional_component.definition = plasmid.identity

new_strain.modules = [chassis_module]
new_strain.functionalComponents = [plasmid_functional_component]

transformation_activity_association = sbol2.Association(
    f"transform_{chassis.name}_association"
)

transformation_activity_plan = sbol2.Plan(f"{new_strain.displayId}_transformation_plan")
transformation_activity_plan.description = (
    "TODO: generate accurate description of transformation"
)
transformation_activity_association.plan = transformation_activity_plan

transformation_activity_agent = sbol2.Agent("BuildCompiler")
transformation_activity_association.agent = transformation_activity_agent

transformation_activity.associations = [transformation_activity_association]

new_strain.wasGeneratedBy = transformation_activity

assembly_doc.add_list(
    [
        transformation_activity,
        chassis,
        chassis_usage,
        chassis_module,
        new_strain,
        plasmid_functional_component,
        transformation_activity_plan,
    ]
)

In [4]:
assembly_doc.write("transformation_activity_new.xml")

'Valid.'

In [5]:
cd_doc = sbol2.Document()
sbh.pull(
    "https://synbiohub.org/user/Gon/abstracttoassemblyplan/composite_1/1/1c2bb5c5e250a63b6c3edae2e25ded344777db3f/share",
    cd_doc,
)

In [6]:
print(cd_doc)

Design........................0
Build.........................0
Test..........................0
Analysis......................0
ComponentDefinition...........13
ModuleDefinition..............0
Model.........................0
Sequence......................13
Collection....................0
Activity......................0
Plan..........................0
Agent.........................0
Attachment....................0
CombinatorialDerivation.......0
Implementation................0
SampleRoster..................0
Experiment....................0
ExperimentalData..............0
Annotation Objects............0
---
Total: .........................26



In [7]:
for cd in cd_doc.componentDefinitions:
    print(f"{cd.displayId}, {len(cd.components)}, {len(cd.sequenceAnnotations)}")

composite_1, 10, 10
Ligation_Scar_A, 0, 0
Ligation_Scar_B, 0, 0
Ligation_Scar_C, 0, 0
Ligation_Scar_D, 0, 0
Ligation_Scar_E, 0, 0
B0015, 0, 0
B0034, 0, 0
E0040m_gfp, 0, 0
J23100, 0, 0
dvk_backbone_core, 2, 2
kan_region, 0, 0
origin_of_replication_pSB1A2, 0, 0


In [8]:
from sbol2build import get_top_level_component_definitions

print([cd.displayId for cd in get_top_level_component_definitions(cd_doc)])

['composite_1']
